### read_csv

file, sep , header, names , index_col, usecols, dtyes, parse_dates
- add duplicate suffix to columns
- header = None
- header = 0 , skip first row cols = [] , pass the colname 

### setting an index 
- parse_dates=True
- usecols , select the columns , saves processing time and memory ( add index also otherwise error)

### select rows 
nrows 
skiprows based on index position, nrows works after skiprows
- skiprows = lambda x:x%2 == 1

### missing values 
- na_values =[".","Missing"] --> NaN